In [1]:
import mdtraj as md
import numpy as np
import os

In [2]:
# inputdir, outputdir
inputdir = "input"

outputdir = "output"
os.makedirs(outputdir, exist_ok=True)

In [3]:
trj = md.load_trr(f"{inputdir}/out.trr", top=f"{inputdir}/solv_ions_prot.gro")
print(trj)

TypeError: 'NoneType' object cannot be interpreted as an integer

In [ ]:
trj.xyz.shape

# make weight list

In [ ]:
from weightdict import atomicWeightsDecimal as wdict
weight_key = 'standard'

atoms_dict = trj.topology.to_dataframe()[0].element

wlist = [float(wdict[atom][weight_key]) for atom in atoms_dict]

# centering

In [ ]:
# before centering
for t in range(10):
    tmpxyz = trj.xyz[t]

    total_x = 0
    for i in range(tmpxyz.shape[0]):
        total_x = total_x + tmpxyz[i][0]*wlist[i]
    
    print(total_x)

In [ ]:
# centering
trj.center_coordinates(mass_weighted=True)

trj.save_trr(f"{outputdir}/test.trr")

In [ ]:
# after centering
for t in range(10):
    tmpxyz = trj.xyz[t]

    total_x = 0
    for i in range(tmpxyz.shape[0]):
        total_x = total_x + tmpxyz[i][0]*wlist[i]
    
    print(total_x)

# choose 2 frames

In [ ]:
a = trj.xyz[0]
b = trj.xyz[-1]

# matrix U

In [ ]:
U = np.empty((3,3))

for i in range(3):
    for j in range(3):
        U[i][j] = sum( [wlist[n]*a[n ,i]*b[n, j] for n in range(a.shape[0])] )

print(U)

# 固有値問題

In [ ]:
# 6*6のomega定義
OMEGA = np.empty((6,6))

for i in range(3):
    for j in range(3):
        OMEGA[i][j] = 0
        OMEGA[i+3][j+3] = 0
        OMEGA[i+3][j] = U[i][j]
        OMEGA[i][j+3] = U.T[i][j]

print(OMEGA)

In [ ]:
# 固有値問題
eig_val, eig_vec =np.linalg.eig(OMEGA)

# pairs
for i in range(len(eig_val)):
    print(eig_val[i])
    print(eig_vec[i])
    print('----------')

In [ ]:
# root2を出す
hks = eig_vec * np.sqrt(2)

h = hks[:, 0:3]
k = hks[:, 3:6]

print(h.shape, k.shape)

# 回転行列R

In [ ]:
R = np.dot(h.T,k)

In [ ]:
print(R)
print(np.linalg.det(R))

# bをrへ変換

In [ ]:
r = np.empty_like(b)

for i in range(3):
    for n in range(r.shape[0]):
        r[n,i] = sum( [R[i,j]*b[n,j] for j in range(3)] )

# save

In [ ]:
fitted_xyz = np.array([a,r])

fitted_xyz.shape

In [ ]:
fitted_trj = md.Trajectory(fitted_xyz, trj.topology)

In [ ]:
fitted_trj.save_trr(f"{outputdir}/test.trr")